### 1. Import libraries

In [6]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

!pip install folium
!pip install geocoder
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### 2. Scrap data from Wikipedia page into a DataFrame 

In [36]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Lucknow").text

In [37]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [78]:
soup

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Category:Neighbourhoods in Lucknow - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"YBZ9BDDN0YaclFJVgiiWkQAAAJY","wgCSPNonce":!1,"wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":14,"wgPageName":"Category:Neighbourhoods_in_Lucknow","wgTitle":"Neighbourhoods in Lucknow","wgCurRevisionId":796836602,"wgRevisionId":796836602,"wgArticleId":18717639,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Lucknow","Neighbourhoods in Uttar Pradesh"],"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgRelev

In [79]:
# create a list to store neighborhood data
neighborhoodList = []

In [80]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [81]:
# create a new DataFrame from the list
lkw_df = pd.DataFrame({"Neighborhood": neighborhoodList})

#lkw_df = kl_df.iloc[1:]
lkw_df

,Neighborhood
0,Aishbagh
1,Alambagh
2,"Aminabad, Lucknow"
3,"Ashiyana, Lucknow"
4,Badshah Nagar
5,Cis-Gomti area
6,Dilkusha
7,Dugawan
8,Gola Ganj
9,Gomti Nagar


In [82]:
lkw_df.shape

(21, 1)

### 3.Get the geographical coordinates

In [75]:
address = 'Lucknow, India'

geolocator = Nominatim(user_agent="CN_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 26.8381, 80.9346001.


In [85]:
# define a function to get coordinates
def get_latlng(neighborhood):
    #print("neighboutrhood", neighborhood)
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Lucknow, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [86]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in lkw_df["Neighborhood"].tolist() ]

In [87]:
coords

[[26.840180000000032, 80.90053000000006],
 [26.814620000000048, 80.90332000000006],
 [26.845420000000047, 80.92722000000003],
 [26.788560000000075, 80.92003000000005],
 [26.86949000000004, 80.96114000000006],
 [26.901576437020168, 80.87401047859508],
 [26.794070000000033, 80.93798000000004],
 [26.845250000000078, 80.92002000000008],
 [26.850450000000023, 80.92582000000004],
 [26.84922000000006, 80.99726000000004],
 [26.85471000000007, 80.92135000000007],
 [26.848380000000077, 80.95307000000008],
 [26.88668000000007, 80.99333000000007],
 [26.79657000000003, 80.88426000000004],
 [26.84761000000003, 80.92226000000005],
 [26.850270000000023, 80.89346000000006],
 [26.840340000000026, 80.91418000000004],
 [26.88196000000005, 80.97083000000003],
 [26.900400000000047, 80.98706000000004],
 [26.85471000000007, 80.92135000000007],
 [26.899890000000028, 80.95233000000007]]

In [88]:
len(coords)

21

In [89]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [93]:
# # merge the coordinates into the original dataframe
lkw_df['Latitude'] = df_coords['Latitude']
lkw_df['Longitude'] = df_coords['Longitude']

# check the neighborhoods and the coordinates
print(lkw_df.shape)
lkw_df

(21, 3)


,Neighborhood,Latitude,Longitude
0,Aishbagh,26.840180,80.90053
1,Alambagh,26.814620,80.90332
2,"Aminabad, Lucknow",26.845420,80.92722
3,"Ashiyana, Lucknow",26.788560,80.92003
4,Badshah Nagar,26.869490,80.96114
5,Cis-Gomti area,26.901576,80.87401
6,Dilkusha,26.794070,80.93798
7,Dugawan,26.845250,80.92002
8,Gola Ganj,26.850450,80.92582
9,Gomti Nagar,26.849220,80.99726


In [94]:
# save the DataFrame as CSV file
lkw_df.to_csv("lkw_df.csv", index=False)

# Create a map of Lucknow with neighborhoods superimposed on top¶

In [97]:
# get the coordinates of Kuala Lumpur
address = 'Lucknow, India'

geolocator = Nominatim(user_agent="LKW_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Lucknow, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Lucknow, India 26.8381, 80.9346001.


In [98]:
# create map of Toronto using latitude and longitude values
map_lkw = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(lkw_df['Latitude'], lkw_df['Longitude'], lkw_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_lkw)  
    
map_lkw

In [99]:
# save the map as HTML file
map_lkw.save('map_lkw.html')

### Use the Foursquare API to explore the neighborhoods

In [100]:
# define Foursquare Credentials and Version
CLIENT_ID = '3I05EYEV3HVMFUAUUEMAHF2T5A21QDJMVWKX4TITMQBJ4XKR' # your Foursquare ID
CLIENT_SECRET = 'CPU3W4GZF5MSNNT2FKXXWZJPE5L4D55D4YHAG4CWTCN3NJ4I' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3I05EYEV3HVMFUAUUEMAHF2T5A21QDJMVWKX4TITMQBJ4XKR
CLIENT_SECRET:CPU3W4GZF5MSNNT2FKXXWZJPE5L4D55D4YHAG4CWTCN3NJ4I


#### Now, let's get the top 100 venues that are within a radius of 2000 meters.

In [101]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(lkw_df['Latitude'], lkw_df['Longitude'], lkw_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [102]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(234, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Aishbagh,26.84018,80.90053,Naka Hindola,26.837176,80.920334,Market
1,Aishbagh,26.84018,80.90053,mubeens BrBQ! A place for enjoying a perfect t...,26.831293,80.891922,BBQ Joint
2,Aishbagh,26.84018,80.90053,Axis Bank ATM,26.831772,80.888654,ATM
3,Aishbagh,26.84018,80.90053,Buniyaad baug,26.856890,80.894330,Park
4,Aishbagh,26.84018,80.90053,Lucknow Junction Railway Station | लखनऊ जंक्शन...,26.831779,80.918341,Train Station


#### Let's check how many venues were returned for each neighorhood

In [103]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Aishbagh,5,5,5,5,5,5
Alambagh,10,10,10,10,10,10
"Aminabad, Lucknow",27,27,27,27,27,27
"Ashiyana, Lucknow",9,9,9,9,9,9
Badshah Nagar,18,18,18,18,18,18
Dilkusha,7,7,7,7,7,7
Dugawan,10,10,10,10,10,10
Gola Ganj,19,19,19,19,19,19
Gomti Nagar,10,10,10,10,10,10


#### Let's find out how many unique categories can be curated from all the returned venues

In [104]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 50 uniques categories.


In [107]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:60]

array(['Market', 'BBQ Joint', 'ATM', 'Park', 'Train Station',
       'Shopping Mall', 'Fast Food Restaurant', 'Pizza Place',
       'Accessories Store', 'Multiplex', 'Bus Station', 'Food Court',
       'Electronics Store', 'Department Store', 'Indian Restaurant',
       'Ice Cream Shop', 'Tea Room', 'Neighborhood', 'Café', 'Hotel',
       'Coffee Shop', 'American Restaurant', 'Restaurant', 'Snack Place',
       'Lounge', 'Business Service', 'Outdoors & Recreation',
       'Clothing Store', 'Sports Bar', 'Asian Restaurant',
       'Chinese Restaurant', 'Bakery', 'Music Venue', 'Gym',
       'Vegetarian / Vegan Restaurant', 'Golf Course', 'Food Truck',
       "Women's Store", 'Hookah Bar', 'Plaza', 'History Museum',
       'Breakfast Spot', 'Antique Shop', 'Sandwich Place',
       'Bed & Breakfast', 'Sculpture Garden', 'Fried Chicken Joint',
       'Flea Market', 'Arcade', 'Building'], dtype=object)

In [111]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()


True

#### Analyze Each Neighborhood

In [112]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(234, 51)


,Neighborhoods,ATM,Accessories Store,American Restaurant,Antique Shop,Arcade,Asian Restaurant,BBQ Joint,Bakery,Bed & Breakfast,Breakfast Spot,Building,Bus Station,Business Service,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Department Store,Electronics Store,Fast Food Restaurant,Flea Market,Food Court,Food Truck,Fried Chicken Joint,Golf Course,Gym,History Museum,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Lounge,Market,Multiplex,Music Venue,Neighborhood,Outdoors & Recreation,Park,Pizza Place,Plaza,Restaurant,Sandwich Place,Sculpture Garden,Shopping Mall,Snack Place,Sports Bar,Tea Room,Train Station,Vegetarian / Vegan Restaurant,Women's Store
0,Aishbagh,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Aishbagh,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Aishbagh,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Aishbagh,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,Aishbagh,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [114]:
#Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(20, 51)


,Neighborhoods,ATM,Accessories Store,American Restaurant,Antique Shop,Arcade,Asian Restaurant,BBQ Joint,Bakery,Bed & Breakfast,Breakfast Spot,Building,Bus Station,Business Service,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Department Store,Electronics Store,Fast Food Restaurant,Flea Market,Food Court,Food Truck,Fried Chicken Joint,Golf Course,Gym,History Museum,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Lounge,Market,Multiplex,Music Venue,Neighborhood,Outdoors & Recreation,Park,Pizza Place,Plaza,Restaurant,Sandwich Place,Sculpture Garden,Shopping Mall,Snack Place,Sports Bar,Tea Room,Train Station,Vegetarian / Vegan Restaurant,Women's Store
0,Aishbagh,0.200000,0.0,0.000000,0.000000,0.000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000
1,Alambagh,0.000000,0.1,0.000000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.100000,0.100000,0.00,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.200000,0.0,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Aminabad, Lucknow",0.000000,0.0,0.037037,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.148148,0.000000,0.000000,0.037037,0.000000,0.000000,0.037037,0.00,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.037037,0.185185,0.000000,0.037037,0.000000,0.000000,0.037037,0.000000,0.000000,0.037037,0.0,0.037037,0.000000,0.000000,0.074074,0.037037,0.000000,0.037037,0.074074,0.000000,0.000000
3,"Ashiyana, Lucknow",0.000000,0.0,0.000000,0.000000,0.000,0.111111,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.111111,0.000000,0.111111,0.111111,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.111111,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000
4,Badshah Nagar,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.0,0.111111,0.000000,0.000000,0.000000,0.0,0.000000,0.055556,0.000000,0.055556,0.111111,0.055556,0.055556,0.055556,0.00,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.055556,0.055556,0.000000,0.055556,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.055556,0.0,0.055556,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000
5,Dilkusha,0.571429,0.0,0.000000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.142857,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857
6,Dugawan,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.00,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.100000,0.200000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000
7,Gola Ganj,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.052632,0.000000,0.000000,0.052632,0.000000,0.000000,0.052632,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.105263,0.157895,0.052632,0.263158,0.00

In [115]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

11

#### Create a new DataFrame for Shopping Mall data only

In [117]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [118]:
kl_mall.head()

,Neighborhoods,Shopping Mall
0,Aishbagh,0.000000
1,Alambagh,0.100000
2,"Aminabad, Lucknow",0.074074
3,"Ashiyana, Lucknow",0.000000
4,Badshah Nagar,0.055556


#### Run k-means to cluster the neighborhoods in Kuala Lumpur into 3 clusters. 

In [123]:
kclusters = 5

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 3, 4, 1, 0, 1, 3, 3, 3, 1], dtype=int32)

In [124]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [125]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Aishbagh,0.000000,1
1,Alambagh,0.100000,3
2,"Aminabad, Lucknow",0.074074,4
3,"Ashiyana, Lucknow",0.000000,1
4,Badshah Nagar,0.055556,0


In [126]:
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(20, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Aishbagh,0.000000,1,NaN,NaN
1,Alambagh,0.100000,3,26.845420,80.92722
2,"Aminabad, Lucknow",0.074074,4,26.788560,80.92003
3,"Ashiyana, Lucknow",0.000000,1,26.869490,80.96114
4,Badshah Nagar,0.055556,0,26.901576,80.87401


In [134]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(18, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
4,Badshah Nagar,0.055556,0,26.901576,80.87401
10,Hazratganj,0.032258,0,26.886680,80.99333
5,Dilkusha,0.000000,1,26.845250,80.92002
9,Hata -e- Sheikhan,0.000000,1,26.848380,80.95307
18,Trans-Gomti area,0.000000,1,26.899890,80.95233
3,"Ashiyana, Lucknow",0.000000,1,26.869490,80.96114
16,Sarvodaya Nagar,0.000000,1,26.900400,80.98706
17,Shivpuri colony,0.000000,1,26.854710,80.92135
14,"Mehndiganj, Uttar Pradesh, Lucknow",0.000000,1,26.840340,80.91418
12,"Krishna Nagar, Lucknow",0.166667,2,26.847610,80.92226


In [133]:
kl_merged = kl_merged[kl_merged['Latitude'].notna()]

In [135]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [136]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

#### Cluster 0

In [137]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
4,Badshah Nagar,0.055556,0,26.901576,80.87401
10,Hazratganj,0.032258,0,26.886680,80.99333


#### Cluster 1

In [139]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
5,Dilkusha,0.0,1,26.84525,80.92002
9,Hata -e- Sheikhan,0.0,1,26.84838,80.95307
18,Trans-Gomti area,0.0,1,26.89989,80.95233
3,"Ashiyana, Lucknow",0.0,1,26.86949,80.96114
16,Sarvodaya Nagar,0.0,1,26.90040,80.98706
17,Shivpuri colony,0.0,1,26.85471,80.92135
14,"Mehndiganj, Uttar Pradesh, Lucknow",0.0,1,26.84034,80.91418


#### Cluster 2

In [140]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
12,"Krishna Nagar, Lucknow",0.166667,2,26.84761,80.92226


#### Cluster 3

In [141]:
kl_merged.loc[kl_merged['Cluster Labels'] == 3]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
8,Gomti Nagar,0.100000,3,26.85471,80.92135
6,Dugawan,0.100000,3,26.85045,80.92582
15,"Rajendra Nagar, Lucknow",0.125000,3,26.88196,80.97083
1,Alambagh,0.100000,3,26.84542,80.92722
7,Gola Ganj,0.105263,3,26.84922,80.99726


#### Cluster 4

In [142]:
kl_merged.loc[kl_merged['Cluster Labels'] == 4]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
13,Maulviganj,0.076923,4,26.85027,80.89346
11,"Indira Nagar, Lucknow",0.090909,4,26.79657,80.88426
2,"Aminabad, Lucknow",0.074074,4,26.78856,80.92003


#### Observations:
Most of the shopping malls are concentrated in the central area of Lucknow city, with the highest number in cluster 3 and moderate number in cluster 4. On the other hand, cluster 1 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 1 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 0 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of shopping malls and suffering from intense competition.